In [1]:
import pandas as pd
import numpy as np
from Fin import *
from Stat import *
from scipy import stats
import statsmodels.api as sm
import warnings

warnings.filterwarnings(action='ignore')

# 필요한 데이터 먼저 테이블로 정렬시키기

In [2]:
Alldata = pd.read_excel("Data/kosdaq/Alldata.xlsx")
UseOnlyPort = Alldata[['Symbol','회계년','Name','시가총액','consen_cut','CompuConsen','valPEIS','deltaprice','PER',\
                      'PBR', 'BETA']]
UseOnlyPort['ACC'] = Alldata['영업이익(천원)'] - Alldata['영업활동으로인한현금흐름(천원)']
UseOnlyPort['HEDGE'] = 0

#  포트폴리오 구성하기
1. 컨센별 포트폴리오 ConsenPorts
2. PEIS별 포트폴리오 PEISPorts
3. Uncondi Hold Table8list
4. condi Hold 마지막 라인~
* 여기서는 Strongbuy/ buy만 쓰면댐 / sell이 없어서 걍 홀드를 sell로 보자

In [3]:
ConsenPorts = MakePortbyCriteria(UseOnlyPort, 'consen_cut')
PEISPorts = MakePortbyPEIS(UseOnlyPort)
for i in ConsenPorts:
    i.ConsenPEIS()
    i.Maketable8() 
# 다중회귀 더 하는 것
Table8list = [PEISPorts[0],PEISPorts[4],ConsenPorts[2]]
Maketable8(Table8list)
ConsenPorts[2].PEIS[0].Maketable8()
ConsenPorts[2].PEIS[4].Maketable8()

# BUY / SELL 전략

In [4]:
MultiLinear(ConsenPorts[3].table8,ConsenPorts[2].table8)

N :  1240


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     AR   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                     10.75
Date:                Fri, 21 May 2021   Prob (F-statistic):           1.11e-11
Time:                        11:15:59   Log-Likelihood:                -985.80
No. Observations:                1240   AIC:                             1986.
Df Residuals:                    1233   BIC:                             2021.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3024      0.101     -2.991      0.003      -0.501      -0.104
HEDGE          0.3113      0.093      3.362      0.001       0.130       0.493
PBR           -0.0060      0.008     -0.782      0.434      -0.021       0.009
RET            0.1919      0.029      6.618      0.000       0.135       0.249
PER        -1.013e-06    1.9e-06     -0.533      0.594   -4.74e-06    2.71e-06
ACC         1.635e-10   3.12e-10      0.524      0.600   -4.49e-10    7.76e-10
BETA           0.1052      0.044      2.405      0.016       0.019       0.191
==============================================================================
Omnibus:                      665.872   Durbin-Watson:                   2.058
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6527.259
Skew:                           2.299   Prob(JB):                         0.00
Kurtosis:                      13.257   Cond. No.                     4.34e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.34e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# fundamental 전략

In [5]:
MultiLinear(PEISPorts[4].table8, PEISPorts[0].table8) 

N :  3339


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     AR   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     11.81
Date:                Fri, 21 May 2021   Prob (F-statistic):           3.79e-13
Time:                        11:15:59   Log-Likelihood:                -3895.0
No. Observations:                3339   AIC:                             7804.
Df Residuals:                    3332   BIC:                             7847.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0966      0.041     -2.372      0.018      -0.176      -0.017
HEDGE          0.2633      0.032      8.126      0.000       0.200       0.327
PBR           -0.0004      0.001     -0.506      0.613      -0.002       0.001
RET            0.0206      0.016      1.268      0.205      -0.011       0.052
PER          4.27e-05   5.11e-05      0.836      0.403   -5.75e-05       0.000
ACC        -9.499e-11   2.08e-10     -0.458      0.647   -5.02e-10    3.12e-10
BETA          -0.0146      0.032     -0.455      0.649      -0.077       0.048
==============================================================================
Omnibus:                     2458.864   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            92905.836
Skew:                           3.081   Prob(JB):                         0.00
Kurtosis:                      28.096   Cond. No.                     2.49e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.49e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Unconditional Hold 전략

In [6]:
MultiLinear(ConsenPorts[2].table8) 

N :  35


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     AR   R-squared:                       0.287
Model:                            OLS   Adj. R-squared:                  0.164
Method:                 Least Squares   F-statistic:                     2.338
Date:                Fri, 21 May 2021   Prob (F-statistic):             0.0670
Time:                        11:15:59   Log-Likelihood:                -27.009
No. Observations:                  35   AIC:                             66.02
Df Residuals:                      29   BIC:                             75.35
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0035      0.285      0.012      0.990      -0.579       0.586
PBR            0.1780      0.144      1.238      0.225      -0.116       0.472
RET           -0.4785      0.191     -2.511      0.018      -0.868      -0.089
PER            0.0003      0.001      0.419      0.678      -0.001       0.002
ACC         1.932e-09   2.65e-09      0.730      0.471   -3.48e-09    7.34e-09
BETA          -0.3549      0.271     -1.308      0.201      -0.910       0.200
==============================================================================
Omnibus:                        7.086   Durbin-Watson:                   2.213
Prob(Omnibus):                  0.029   Jarque-Bera (JB):                5.575
Skew:                          -0.881   Prob(JB):                       0.0616
Kurtosis:                       3.848   Cond. No.                     1.58e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.58e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# conditional Hold 전략

In [7]:
MultiLinear(ConsenPorts[2].PEIS[4].table8, ConsenPorts[2].PEIS[0].table8) 

N :  13


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     AR   R-squared:                       0.466
Model:                            OLS   Adj. R-squared:                  0.085
Method:                 Least Squares   F-statistic:                     1.222
Date:                Fri, 21 May 2021   Prob (F-statistic):              0.390
Time:                        11:15:59   Log-Likelihood:                -3.0956
No. Observations:                  13   AIC:                             18.19
Df Residuals:                       7   BIC:                             21.58
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1033      0.457      0.226      0.828      -0.977       1.184
PBR           -0.4781      0.304     -1.575      0.159      -1.196       0.240
RET            0.2847      0.274      1.039      0.334      -0.363       0.933
PER            0.0013      0.004      0.331      0.750      -0.008       0.011
ACC         2.052e-09   3.28e-09      0.625      0.552   -5.71e-09    9.81e-09
BETA           0.4594      0.523      0.879      0.409      -0.777       1.696
==============================================================================
Omnibus:                        1.077   Durbin-Watson:                   2.235
Prob(Omnibus):                  0.583   Jarque-Bera (JB):                0.701
Skew:                          -0.009   Prob(JB):                        0.704
Kurtosis:                       1.862   Cond. No.                     3.10e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.1e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""